In [ ]:
import urllib
import datetime
import json

In [ ]:
# [code1] url 접속을 요청하고 응답을 받아서 반환
def getRequestUrl(url):
    req = urllib.request.Request(url)  #매개변수로 받은 url에 대한 요청을 보낼 객체를 생성
    try:
        response = urllib.request.urlopen(req)  #요청 객체를 보내서 받은 응답 데이터를 response 객체에 저장

        #response 객체에 저장된 코드를 확인. 코드가 200이면 요청을 정상 처리한 것이므로 성공.
        # 메시지와 현재 시간을 파이썬 셸 창에 출력하고 응답을 utf-8 형식으로 디코딩하여 반환.
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None


In [ ]:
# [code2] 입국관광통계서비스의 오픈 API를 사용하여 데이터 요청 url을 만들고 [CODE 1]의 getRequestUrl(url)을 호출해서 받은 응답 데이터를 반환

def getTourismStatsItem(yyyymm, nat_cd, ed_cd):
    service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"


    parameters = "?_type=json&serviceKey=" + 'TEo5fURVg6O3ChvfXOmzkr0IXbTl0d4VkfIj3JVTz0ctJ%2BNS0IjPHxLXlijxDlubeXvzd3ZlGksTn%2FHhACp8gA%3D%3D' #인증키
    parameters += "&YM=" + yyyymm
    parameters += "&NAT_CD=" + nat_cd
    parameters += "&ED_CD=" + ed_cd

    url = service_url + parameters
    print(url)  #액세스 거부 여부 확인용 출력
    responseDecode = getRequestUrl(url)  #[CODE 1]

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)


In [ ]:
# [code3] 수집 기간 동안 월 단위로 [CODE 2]의 getTourismStatsItem()을 호출해 받은 데이터를 리스트로 묶어 반환

def getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear):
    jsonResult = []
    result = []
    for year in range(nStartYear, nEndYear + 1):
        for month in range(1, 13):
            yyyymm = "{0}{1:0>2}".format(str(year), str(month))
            jsonData = getTourismStatsItem(yyyymm, nat_cd, ed_cd)  #[CODE 2]
            if (jsonData['response']['header']['resultMsg'] == 'OK'):
                #데이터가 없는 마지막 항목인 경우 ----------------------------
                if jsonData['response']['body']['items'] == '':
                    dataEND = "{0}{1:0>2}".format(str(year), str(month - 1))
                    print("데이터 없음.... \n제공되는 통계 데이터는 %s년 %s월까지입니다."
                          % (str(year), str(month - 1)))
                    break
                #jsonData를 출력하여 확인............................................
                print(json.dumps(jsonData, indent=4,
                                 sort_keys=True, ensure_ascii=False))

                natName = jsonData['response']['body']['items']['item']
                ['natKorNm']
            natName = natName.replace(' ', '') #띄어쓰기 제거
            num = jsonData['response']['body']['items']['item']['num']
            ed = jsonData['response']['body']['items']['item']['ed']
            print('[ %s_%s : %s ]' % (natName, yyyymm, num))
            print('------------------------------------------------------')
            jsonResult.append({'nat_name': natName, 'nat_cd': nat_cd,
                               'yyyymm': yyyymm, 'visit_cnt': num})
            result.append([natName, nat_cd, yyyymm, num])
    return (jsonResult, result, natName, ed, dataEND)

